# Spam/Jailbreak Classification

**Fine-Tuning a Base BERT Model**

---

## Dependencies

### Modules

In [ ]:
%pip install fastai

In [ ]:
%pip install torch

In [ ]:
%pip install transformers

In [ ]:
%pip install datasets

In [ ]:
%pip install tokenizers

In [ ]:
%pip install scikit-learn

In [ ]:
%pip install matplotlib

In [ ]:
%pip install spacy

In [ ]:
%pip install evaluate

In [ ]:
%pip install accelerate

### Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from fastai.text.all import *
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding, pipeline
from datasets import Dataset, DatasetDict
import numpy as np
import evaluate
import os
from sklearn.metrics import precision_recall_fscore_support

/Users/admin/Desktop/Spam-Jailbreak-Classification/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


---

## Data

190K+ Spam | Ham Email Dataset for Classification: https://www.kaggle.com/datasets/meruvulikith/190k-spam-ham-email-dataset-for-classification

Emails for spam or ham classification (Trec 2007): https://www.kaggle.com/datasets/bayes2003/emails-for-spam-or-ham-classification-trec-2007?select=email_text.csv

### Filtering

In [4]:
directory = "data"
try:
    os.mkdir(directory)
    print(f"Directory '{directory}' created successfully.")
except FileExistsError:
    print(f"Directory '{directory}' already exists.")

Directory 'data' created successfully.


Download 2 datasets into "data" folder

In [5]:
# df_a = pd.read_csv("data/email_text.csv")
# df_b = pd.read_csv("data/spam_Emails_data.csv")

#kaggle 
df_a = pd.read_csv("/kaggle/input/emails-for-spam-or-ham-classification-trec-2007/email_text.csv")
df_b = pd.read_csv("/kaggle/input/190k-spam-ham-email-dataset-for-classification/spam_Emails_data.csv")

In [8]:
df_a

,label,text
0,1,do you feel the pressure to perform and not rising to the occasion try v ia gr a your anxiety will be a thing of the past and you will be back to your old self
1,0,hi i've just updated from the gulus and i check on other mirrors it seems there is a little typo in debian readme file example http gulus usherbrooke ca debian readme ftp ftp fr debian org debian readme testing or lenny access this release through dists testing the current tested development snapshot is named etch packages which have been tested in unstable and passed automated tests propogate to this release etch should be replace by lenny like in the readme html yan morin consultant en logiciel libre yan morin savoirfairelinux com escapenumber escapenumber escapenumber to unsubscribe ema...
2,1,mega authenticv i a g r a discount pricec i a l i s discount pricedo not miss it click here http www moujsjkhchum com
3,1,hey billy it was really fun going out the other night and talking while we were out you said that you felt insecure about your manhood i noticed in the toilets you were quite small in that area but not to worry that website that i was telling you about is my secret weapon to an extra escapenumber inches trust me girls love bigger ones i've had escapenumber times as many chicks since i used these pills a year ago the package i used was the escapenumber month supply one and its worth every cent and more the website is http ctmay com ring me on the weekend and we will go out and drink again a...
4,1,system of the home it will have the capabilities to be linked far as i know and what i am doing within it as a part of it so with respect to the affects of technology on society we have science ad agencies are cashin g in on its' commerciality and photographs and paint on electronic canvases it still seems like silence white out black out lights out it didn't happen although far from p erfect especially in that it precludes a vast explanation for this i can' t understand how people can rely so avant gardes of the art world additionally writers and lawyers yet to re ach its full potential i...
...,...,...
53663,1,versuchen sie unser produkt und sie werden fuhlen was unsere kunden bestatigen preise die keine konkurrenz kennen kein peinlicher arztbesuch erforderlicht kein langes warten auslieferung innerhalb von escapenumber escapenumber tagen diskrete verpackung und zahlung kostenlose arztliche telefon beratung bequem und diskret online bestellen visa verifizierter onlineshop keine versteckte kosten nur fur kurze zeit vier pillen umsonst erhalten http osxqpu setcare hk escapenumber
53664,1,while we may have high expectations of our associates we also give them high rewards imagine being part of a stable organization with a sterling reputation a place where the sydney car centre is an integral part of all that we do with our car centre personality you'll not just succeed you'll thrive and with our strong commitment to promoting from within you'll definitely enjoy your rise to the top today the sydney car centre is looking for an industrious regional assistant to fasten the process of the delivery of customer payments to the suppliers the position offered is a part time job an...
53665,0,for those who are interested i just cook a little tcltkhelp function to ease access to the tcl tk documentation under windows this is on the wiki discussion of the tkcommands help page at http wiki r project org rwiki doku php id rdoc tcltk tkcommands best philippe grosjean ° prof philippe grosjean numerical ecology of aquatic systems mons hainaut university belgium duncan murdoch wrote on escapenumber escapenumber escapenumber escapenumber escapenumber am mike meredith wrote i think it would help if the tcl tk manuals were added to the rgui help menu why google when they are on your hard ...
53666,0,hello as i wrote i call sqlfetch channel t studie and this function generates in the background the concrete query how can i view log the concrete query regards bernhard

In [6]:
df_b['label'] = df_b['label'].map({"Spam": 1, "Ham": 0})

In [7]:
df_b['label'].unique()

array([1, 0])

In [8]:
merged = pd.concat([df_a[['label', 'text']], df_b[['label', 'text']]], ignore_index=True)
merged

,label,text
0,1,do you feel the pressure to perform and not rising to the occasion try v ia gr a your anxiety will be a thing of the past and you will be back to your old self
1,0,hi i've just updated from the gulus and i check on other mirrors it seems there is a little typo in debian readme file example http gulus usherbrooke ca debian readme ftp ftp fr debian org debian readme testing or lenny access this release through dists testing the current tested development snapshot is named etch packages which have been tested in unstable and passed automated tests propogate to this release etch should be replace by lenny like in the readme html yan morin consultant en logiciel libre yan morin savoirfairelinux com escapenumber escapenumber escapenumber to unsubscribe ema...
2,1,mega authenticv i a g r a discount pricec i a l i s discount pricedo not miss it click here http www moujsjkhchum com
3,1,hey billy it was really fun going out the other night and talking while we were out you said that you felt insecure about your manhood i noticed in the toilets you were quite small in that area but not to worry that website that i was telling you about is my secret weapon to an extra escapenumber inches trust me girls love bigger ones i've had escapenumber times as many chicks since i used these pills a year ago the package i used was the escapenumber month supply one and its worth every cent and more the website is http ctmay com ring me on the weekend and we will go out and drink again a...
4,1,system of the home it will have the capabilities to be linked far as i know and what i am doing within it as a part of it so with respect to the affects of technology on society we have science ad agencies are cashin g in on its' commerciality and photographs and paint on electronic canvases it still seems like silence white out black out lights out it didn't happen although far from p erfect especially in that it precludes a vast explanation for this i can' t understand how people can rely so avant gardes of the art world additionally writers and lawyers yet to re ach its full potential i...
...,...,...
247515,0,on escapenumber escapenumber escapenumber rob dixon wrote snip for my elem doc firstchild elem elem elem nextsibling snip i covered that in my earlier email weirder stuff that you only tend to see people coming from a c background do for my node head node node node next but in perl it is rarely necessary to do this sort of loop since most functions return a list that can be iterated over using for for my node head nodes in this case your module should include a children method for my elem doc children or a true iterator my iter doc child iter while my elem iter next if it doesn't then bug ...
247516,1,we have everything you need escapelong cialescapenumbers sescapenumberft tescapenumberbs vescapenumberagra sescapenumberft tescapenumberbs cialescapenumbers vescapenumberagra levescapenumbertra propecescapenumbera valescapenumberum xanescapenumberx ambescapenumberen zybescapenumbern atarescapenumberx atescapenumbervan carescapenumbersoma ultrescapenumberm escapelong lipescapenumbertor merescapenumberdia zocescapenumberr nescapenumberrvasc we respect your privacy we guarantee you a total anonymity of your escapenumberrder visit us escapelong inc online at http www electioo com
247517,0,hi quick question say i have a date variable in a data frame or matrix and i'd like to preserve the date format when using write table however when i export the data i get the generic number underlying the date not the date per se and a number such as escapenumber escapenumber etc are not meaningful in excel is there any way i can preserve the format of a date on writing into a text file tia and best tir r help stat math ethz ch mailing list https stat ethz ch mailman listinfo r help please do read the posting guide http www r project org posting guide html and provide commented minimal se...
247518,1,thank you for your loan request which we recieved on escape

In [9]:
merged['label'].value_counts()

label
0    125905
1    121615
Name: count, dtype: int64

In [10]:
merged['label'].unique()

array([1, 0])

In [11]:
merged['text'].unique().shape

(193849,)

In [12]:
merged = merged.drop_duplicates().reset_index(drop=True)

In [13]:
merged = merged.dropna(subset=['text']).reset_index(drop=True)
merged = merged.drop_duplicates(subset=['text']).reset_index(drop=True)

In [20]:
merged.to_csv("data/merged_spam_ham.csv", index=False)

### Splitting

In [15]:
train_val, test = train_test_split(
    merged,
    train_size=0.9,
    stratify=merged['label'],  
    shuffle=True,
)
train, val = train_test_split(
    train_val,
    train_size=0.8,
    stratify=train_val['label'],  
    shuffle=True,
)

train = train.reset_index(drop=True) #72%
val = val.reset_index(drop=True) #18%
test = test.reset_index(drop=True) #10%


In [16]:
directory = "filtered_data"
try:
    os.mkdir(directory)
    print(f"Directory '{directory}' created successfully.")
except FileExistsError:
    print(f"Directory '{directory}' already exists.")

Directory 'filtered_data' created successfully.


In [ ]:
train.to_csv("filtered_data/spam_ham_train.csv", index=False)
val.to_csv("filtered_data/spam_ham_val.csv", index=False)
test.to_csv("filtered_data/spam_ham_test.csv", index=False)

---

In [78]:
train = pd.read_csv("filtered_data/spam_ham_train.csv")
val = pd.read_csv("filtered_data/spam_ham_val.csv")
test = pd.read_csv("filtered_data/spam_ham_test.csv")

## Classifier

In [ ]:
model_name = "bert-base-uncased" #test cased/uncased
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

In [79]:
#freeze all parameters (weights)
for name, param in model.base_model.named_parameters():
    param.requires_grad = False
#unfreezzing pooler layer's parameters, 
#fine-tuning during training while keeping the rest of the model fixed.
for name, param in model.base_model.named_parameters():
    if "pooler" in name:
        param.requires_grad = True

In [82]:
reduced_train = train
reduced_val = val
reduced_test = test

In [83]:
train_ds = Dataset.from_pandas(reduced_train)
val_ds = Dataset.from_pandas(reduced_val)
test_ds = Dataset.from_pandas(reduced_test)

In [84]:
dataset_dict = DatasetDict({"train": train_ds, "val": val_ds, "test": test_ds})

In [85]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [ ]:
tokenized_data = dataset_dict.map(preprocess_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [87]:
accuracy = evaluate.load("accuracy")

Precision measures how many of the emails predicted as spam (positive class) are actually spam. In other words, it tells us how accurate the model's predictions for spam are.

$Precision= \frac{TP} {TP + FP}$

Recall measures how many of the actual spam emails are correctly identified by the model as spam. It tells us how well the model is able to catch spam emails.

$Recall= \frac{TP} {TP + FN}$

The F1 score is the harmonic mean of precision and recall. It provides a balance between precision and recall, especially when there's a trade-off between the two metrics.

$F1 Score= 2 * \frac{Precision * Recall} {Precision + Recall}$

In [88]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    accuracy_res = accuracy.compute(predictions=predictions, references=labels)

    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, pos_label=1, average='binary')
    
    
    return {
        "accuracy": accuracy_res["accuracy"],
        "precision": precision,
        "recall": recall,
        "f1": f1,
    }

In [ ]:
lr = 1e-4
batch_sz = 32
epoch = 3
wd = 0.01

training_args = TrainingArguments(
    output_dir="bert-spam-ham-classifier-full_dataset",
    per_device_train_batch_size=batch_sz,
    per_device_eval_batch_size=batch_sz,
    num_train_epochs=epoch,
    eval_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,

    learning_rate= lr,
    weight_decay= wd,
    report_to="none"
)

In [90]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["val"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

/tmp/ipykernel_36/1459056268.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [91]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.131100,0.131930,0.952598,0.969580,0.928926,0.948818
2,0.121700,0.122500,0.957327,0.959990,0.949346,0.954638
3,0.118600,0.116966,0.957327,0.963111,0.946013,0.954486


TrainOutput(global_step=13086, training_loss=0.1238428879891917, metrics={'train_runtime': 8741.3023, 'train_samples_per_second': 47.9, 'train_steps_per_second': 1.497, 'total_flos': 1.1014841754932736e+17, 'train_loss': 0.1238428879891917, 'epoch': 3.0})

In [ ]:
!pip3 install tqdm==4.62.1

### Testing

In [92]:
test_results = trainer.predict(tokenized_data["test"])

In [94]:
test_results.metrics

{'test_loss': 0.11436181515455246,
 'test_accuracy': 0.957286561774568,
 'test_precision': 0.9620028802481445,
 'test_recall': 0.9471043734322172,
 'test_f1': 0.9544954935150582,
 'test_runtime': 310.0011,
 'test_samples_per_second': 62.532,
 'test_steps_per_second': 1.955}

In [96]:
model_testing_path = "/kaggle/working/bert-spam-ham-classifier-full_dataset/checkpoint-13086"
model_test = AutoModelForSequenceClassification.from_pretrained(model_testing_path)
tokenizer_testing = AutoTokenizer.from_pretrained(model_testing_path)

In [97]:
clf = pipeline("text-classification", model=model_test, tokenizer=tokenizer_testing)

Device set to use cuda:0


In [100]:
text = "Your session expired. Click here to sign in again."
prediction = clf(text)

prediction

[{'label': 'LABEL_1', 'score': 0.6062796711921692}]

In [103]:
text = "Congratulations! You are the lucky winner of a $1,000 gift card to Amazon! This incredible prize is just a click away.\
To claim your prize, all you need to do is follow the instructions below:\
Click on the link to claim your prize: [Claim My $1,000 Gift Card]\
Fill out your details on the next page.\
Complete a quick survey to verify your information.\
But hurry! This offer is available for a limited time only. You must claim your prize within the next 24 hours to avoid missing out.\
If you have any questions, feel free to reach out to our support team. Don’t miss out on this exclusive opportunity!\
Warm regards,\
Your Amazon Prize Team"
prediction = clf(text)

prediction

[{'label': 'LABEL_1', 'score': 0.9921298623085022}]

In [107]:
text = "Dear [Recipient],\
We hope you’re enjoying the benefits of your Premium Subscription with us. This is a quick reminder that your subscription will automatically renew in the next 7 days.\
To ensure uninterrupted service, the renewal payment of $49.99 will be processed on [Renewal Date]. If you’d like to update your payment information or make any changes to your subscription, please visit your account page here:\
[Update My Subscription]\
If you no longer wish to continue with your subscription, you can cancel it at any time before the renewal date, and no further charges will apply. Simply go to your account settings, and follow the instructions to cancel.\
If you have any questions or need assistance, our support team is here to help. Feel free to contact us at [support@company.com].\
Thank you for choosing [Company Name], and we look forward to continuing to serve you!\
Best regards,\ [Company Name] Team Customer Support"
prediction = clf(text)

prediction

[{'label': 'LABEL_1', 'score': 0.9177644848823547}]

#### Testing learning rate

In [ ]:
results_lr = []

batch_sz = 32
epoch = 3
learning_rates = [1e-5, 2e-5, 3e-5, 5e-5, 1e-4]
for lr in learning_rates:
    print(f"Testing learning rate: {lr}")
    
    training_args = TrainingArguments(
        output_dir="bert-spam-ham-classifier-testing-learning-rate",
        per_device_train_batch_size=batch_sz,
        per_device_eval_batch_size=batch_sz,
        num_train_epochs=epoch,
        eval_strategy="epoch",
        logging_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
    
        learning_rate= lr,
        report_to="none"
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_data["train"],
        eval_dataset=tokenized_data["val"],
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics
    )

    trainer.train()
    test_results = trainer.predict(tokenized_data["test"])

    results_lr.append({"learning_rate": lr, 
                    "loss": test_results.metrics["test_loss"],
                   "accuracy": test_results.metrics["test_accuracy"]})



print("----------------------")
results_lr = sorted(results_lr, key=lambda x: x["accuracy"], reverse=True)
print(results_lr)
print("----------------------")

#### Testing weight decay

In [ ]:
results_wd = []
lr = 1e-4
batch_sz = 32
epoch = 3
weight_decay_values = [0.0, 0.01, 0.05, 0.1, 0.2]
for wd in weight_decay_values:
    print(f"Testing weight decay: {wd}")
    
    training_args = TrainingArguments(
        output_dir="bert-spam-ham-classifier-testing-learning-rate",
        per_device_train_batch_size=batch_sz,
        per_device_eval_batch_size=batch_sz,
        num_train_epochs=epoch,
        eval_strategy="epoch",
        logging_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
    
        learning_rate= lr,
        weight_decay=wd,
        report_to="none"
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_data["train"],
        eval_dataset=tokenized_data["val"],
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics
    )

    trainer.train()
    test_results = trainer.predict(tokenized_data["test"])

    results_wd.append({"weight_decay": wd, 
                    "loss": test_results.metrics["test_loss"],
                   "accuracy": test_results.metrics["test_accuracy"]})

In [ ]:
print("----------------------")
results_wd = sorted(results_wd, key=lambda x: x["accuracy"], reverse=True)
for res in results_wd:
    print(res)
print("----------------------")